In [1]:
# 引用套件
import boto3

# 建立與 sqs 的連線
client = boto3.client(
    'sqs',
    endpoint_url = 'http://cc104.sqs.local:9324',
    region_name = 'dummy_region',
    aws_access_key_id = 'dummy_access_key',
    aws_secret_access_key = 'dummy_secret_key',
    verify = False)

In [2]:
# 建立 queue
client.create_queue(QueueName='test_queue')

{'QueueUrl': 'http://cc104.sqs.local:9324/queue/test_queue',
 'ResponseMetadata': {'RequestId': '00000000-0000-0000-0000-000000000000',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'akka-http/10.1.3',
   'date': 'Sun, 13 Jan 2019 04:03:59 GMT',
   'content-type': 'text/plain; charset=UTF-8',
   'content-length': '429'},
  'RetryAttempts': 0}}

In [3]:
# 調用 test_queue
# get a list of queues, we get back a dict with 'QueueUrls' as a key with a list of queue URLs
# we filter to narrow down the list
queues = client.list_queues(QueueNamePrefix='test_queue')
test_queue_url = queues['QueueUrls'][0]
print(test_queue_url)

http://cc104.sqs.local:9324/queue/test_queue


In [4]:
# 寫入訊息
# send 10 messages to this queue
for i in range(0,10):
    # we set a simple message body for each message
    # for FIFO queues, a 'MessageGroupId' is required, which is a 128 char alphanumeric string
    enqueue_response = client.send_message(QueueUrl=test_queue_url, MessageBody='This is test message #'+str(i))
    # the response contains MD5 of the body, a message Id, MD5 of message attributes, and a sequence number (for FIFO queues)
    print('Message ID : ',enqueue_response['MessageId'])

Message ID :  b0737f23-bdae-4392-9b3f-8ce4f6c80a65
Message ID :  535f81f3-9665-40f9-a0d7-4e1eecf56a3a
Message ID :  faffd731-15e3-4b9c-aa6d-aa14916fe45e
Message ID :  9b25763a-7d20-4756-9ceb-223aaa95948c
Message ID :  a7633443-e354-4d53-b6bf-0dc2eb13ba69
Message ID :  26caa83b-b129-4a2b-887a-5875e42b4450
Message ID :  9a78c287-ecc5-44e8-9b1e-0f9d02fd6284
Message ID :  97776c3d-4d1e-4276-95c2-932f9a6fdb84
Message ID :  3897bd6b-3450-4ad4-829c-22750d6c24d6
Message ID :  3684dc54-6423-4ce6-8505-3d6ccc7e6424


In [5]:
# 刪除訊息
# next, we dequeue these messages - 10 messages at a time (SQS max limit) till the queue is exhausted.
# in production/real setup, I suggest using long polling as you get billed for each request, regardless of an empty response
while True:
    messages = client.receive_message(QueueUrl=test_queue_url,MaxNumberOfMessages=10) # adjust MaxNumberOfMessages if needed
    if 'Messages' in messages: # when the queue is exhausted, the response dict contains no 'Messages' key
        for message in messages['Messages']: # 'Messages' is a list
            # process the messages
            print(message['Body'])
            # next, we delete the message from the queue so no one else will process it again
            client.delete_message(QueueUrl=test_queue_url,ReceiptHandle=message['ReceiptHandle'])
    else:
        print('Queue is now empty')
        break

This is test message #0
This is test message #1
This is test message #2
This is test message #3
This is test message #4
This is test message #5
This is test message #6
This is test message #7
This is test message #8
This is test message #9
Queue is now empty


In [7]:
# 刪除 test queue
# delete queue
client.delete_queue(QueueUrl=test_queue_url)

{'ResponseMetadata': {'RequestId': '00000000-0000-0000-0000-000000000000',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'akka-http/10.1.3',
   'date': 'Sun, 13 Jan 2019 03:57:45 GMT',
   'content-type': 'text/plain; charset=UTF-8',
   'content-length': '250'},
  'RetryAttempts': 0}}